In [3]:
pip install pandas transformers


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/DL SIR/CSV FILES/aHXxveJTJoE.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(100, 2)


In [6]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': '1', 'score': 0.5374414324760437},
 {'label': '2', 'score': 0.8236260414123535},
 {'label': '2', 'score': 0.38360804319381714},
 {'label': '3', 'score': 0.5229857563972473},
 {'label': '2', 'score': 0.8475872278213501},
 {'label': '2', 'score': 0.8814426064491272},
 {'label': '1', 'score': 0.6006845831871033},
 {'label': '2', 'score': 0.836517870426178},
 {'label': '2', 'score': 0.6725406646728516},
 {'label': '1', 'score': 0.5385937094688416},
 {'label': '2', 'score': 0.760703444480896},
 {'label': '2', 'score': 0.3729330003261566},
 {'label': '2', 'score': 0.5169102549552917},
 {'label': '1', 'score': 0.4445331394672394},
 {'label': '1', 'score': 0.6139925122261047},
 {'label': '1', 'score': 0.5367032885551453},
 {'label': '2', 'score': 0.6429020166397095},
 {'label': '2', 'score': 0.5035576224327087},
 {'label': '0', 'score': 0.9186583161354065},
 {'label': '1', 'score': 0.523358166217804},
 {'label': '1', 'score': 0.5088768601417542},
 {'label': '2', 'score': 0.397908061

In [8]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,"<a href=""https://www.youtube.com/watch?v=aHXxv...",1,0.537441
1,Here is an idea that could have happened after...,2,0.823626
2,Blue fle,2,0.383608
3,Doc Hoppper Defeats,3,0.522986
4,Max is such a sweet guy,2,0.847587
...,...,...,...
95,Trading Kings,4,0.346788
96,I love Jim Henson&#39;s cameo as one of Doc Ho...,2,0.917378
97,Hopper’s a dick. He never got what he deserved.,2,0.801664
98,"Top 10 anime climaxes <a href=""https://www.you...",1,0.607109


In [9]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         1            joy
1         2           love
2         2           love
3         3          anger
4         2           love
..      ...            ...
95        4           fear
96        2           love
97        2           love
98        1            joy
99        2           love

[100 rows x 2 columns]


<ipython-input-9-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [10]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         1            joy
1         2           love
2         2           love
3         3          anger
4         2           love
..      ...            ...
95        4           fear
96        2           love
97        2           love
98        1            joy
99        2           love

[100 rows x 2 columns]


In [11]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         1            joy
1         2           love
2         2           love
3         3          anger
4         2           love
..      ...            ...
95        4           fear
96        2           love
97        2           love
98        1            joy
99        2           love

[100 rows x 2 columns]


In [12]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_aHXxveJTJoE_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,"<a href=""https://www.youtube.com/watch?v=aHXxv...",1,0.537441,joy
1,Here is an idea that could have happened after...,2,0.823626,love
2,Blue fle,2,0.383608,love
3,Doc Hoppper Defeats,3,0.522986,anger
4,Max is such a sweet guy,2,0.847587,love
...,...,...,...,...
95,Trading Kings,4,0.346788,fear
96,I love Jim Henson&#39;s cameo as one of Doc Ho...,2,0.917378,love
97,Hopper’s a dick. He never got what he deserved.,2,0.801664,love
98,"Top 10 anime climaxes <a href=""https://www.you...",1,0.607109,joy
